## 小說爬蟲

In [1]:
# -*- coding: utf-8 -*
import requests
from bs4 import BeautifulSoup
import types
r = requests.get("https://8book.com/books/novelbook_98524.html")

soup = BeautifulSoup(r.text, "html.parser")

#print(soup)
link = []
for table in soup.find_all("table", {'class' : 'episodelist'}):
    for tr in table.find_all('tr'):
        for a in tr.find_all('a', href=True):
            link.append('https://8book.com' + a['href'])
novel=''
for i in range(len(link)):
    r = requests.get(link[i])
    r.encoding = 'big5'
    soup = BeautifulSoup(r.text, "html.parser")
    st = soup.find('p')
    
    for s in st:
        novel += str(s).replace('(adsbygoogle = window.adsbygoogle || []).push({});', '')
f = open('novel/novel_1.txt','wb')
#f.write(novel.encode('utf-8'))
f.write(novel.replace('<br/>', '').encode('utf-8'))
f.close()

## 切開字詞

In [2]:
import jieba 
import codecs
import os
from argparse import ArgumentParser

def main():
    f = codecs.open('novel/novel_{0}.txt'.format(2), 'r', 'utf-8')
    textseg=[]
    text = ''
    jieba.load_userdict("dict/user_dict_{0}".format(1))
    seg = jieba.cut(f.readlines()[0],cut_all=False)
    for s in seg:
        textseg.append(s)
    fileseg ='segphase/segphases_{0}.txt'.format(2)
    try:
        os.path.isfile(fileseg)
    except:
        open(fileseg,'wb')
    with open(fileseg,'wb') as fW:
        for i in range(len(textseg)):
            fW.write(textseg[i].encode('utf-8'))
            fW.write('\n'.encode('utf-8'))


main()


Building prefix dict from the default dictionary ...
Loading model from cache C:\Users\user\AppData\Local\Temp\jieba.cache
Loading model cost 0.750 seconds.
Prefix dict has been built succesfully.


## word to vector + 文字雲

In [3]:
from gensim.models import word2vec
import jieba
import logging
from argparse import ArgumentParser
from sklearn.decomposition import PCA
from matplotlib import pyplot
import numpy as np
import matplotlib.pyplot as plt
from wordcloud import WordCloud

def main():

    logging.basicConfig(format='%(asctime)s : %(levelname)s : %(message)s', level=logging.INFO)
    sentences = word2vec.LineSentence('segphase/segphases_{0}.txt'.format(2))
    model = word2vec.Word2Vec(sentences, size=100, alpha=0.0005,min_alpha=0.00001, min_count=20, iter=10)
    model.save("word2vec/word2vec_{0}.model".format(2))
def test():
    model = word2vec.Word2Vec.load('word2vec/word2vec_{0}.model'.format(2))
    #phase = input("Please input a phase: ")
    out = model.wv.similar_by_vector(['郭靖'],topn=10)
    word = []
    # for i in range(args.topn):
    #     word.append(" ".join(out[i][0]))
    try :
        os.path.isfile('tags.txt')
    except :
        open('tags.txt','wb')
    f = open('tags.txt', 'wb')
    for i in range(10):
        f.write(out[i][0]+" ")
        print(out[i][0])
    f.close()
    # phase2 = input("Please input another phase")
    # print(model.similarity(phase, phase2))
    # X = model[model.wv.vocab]
    # pca = PCA(n_components=2)
    # result = pca.fit_transform(X)

    # pyplot.scatter(result[:, 0], result[:, 1])
    # words = list(model.wv.vocab)
    # for i, word in enumerate(words):
    #     pyplot.annotate(word.encode('utf-8'), xy=(result[i, 0], result[i, 1]))
    # pyplot.show()

    # 讀取每首歌的前10個tags

    text = open('tags.txt').read()
    print(text)
    # 設定停用字(排除常用詞、無法代表特殊意義的字詞)
    # stopwords = {}.fromkeys(["書"])
    # 產生文字雲
    wc = WordCloud(font_path="NotoSerifCJKtc-Black.otf", #設置字體
               background_color="white", #背景顏色
               max_words = 2000 ,)      #停用字詞
    wc.generate(text)
    # 視覺化呈現
    plt.imshow(wc)
    plt.axis("off")
    # plt.figure(figsize=(10,6), dpi = 100)
    plt.show()


if __name__ == '__main__':
    #parser = ArgumentParser()
    #parser.add_argument("-n", "--novel", default=1,type=int)
    #parser.add_argument("-m", "--mode", default='train', type=str)
    #parser.add_argument("-t", "--topn", default=10, type=int)
    #args = parser.parse_args()
    #main()
    test()

F:\Anaconda\envs\py36\lib\site-packages\smart_open\ssh.py:34: UserWarning: paramiko missing, opening SSH/SCP/SFTP paths will be disabled.  `pip install paramiko` to suppress
  warnings.warn('paramiko missing, opening SSH/SCP/SFTP paths will be disabled.  `pip install paramiko` to suppress')
F:\Anaconda\envs\py36\lib\site-packages\gensim\utils.py:1197: UserWarning: detected Windows; aliasing chunkize to chunkize_serial
  warnings.warn("detected Windows; aliasing chunkize to chunkize_serial")


ValueError: not enough values to unpack (expected 2, got 1)

In [91]:
'''角色取得'''

character = {1: [], 2: []}

for index in range(len(character)):
    f = codecs.open('user_dict_{0}.bin'.format(index + 1), 'r', 'utf-8')
    a = [item for item in f.readlines()]

    roles = []
    for i in range(len(a)):
        role = ''
        for j in range(len(a[i])):
            if a[i][j] == ' ':
                break
            role = role + a[i][j]
        roles.append(role)
    character[index + 1] = roles
    f.close


print(character)



{1: ['柯景騰', '沈佳儀', '阿和', '許博淳', '廖英宏'], 2: ['阿拓', '思螢', '老板娘', '阿不思']}


In [104]:
import csv
import pandas as pd

mat = {1: {}, 2: {}}

'''corelation matrix'''
for index in range(len(mat)):
    with open('matrix_{0}.csv'.format(index + 1), newline='') as csvfile:
        rows = csv.reader(csvfile)
        row = []
        for i in rows:
            row.append(i)
            
    for i in range(len(character[index + 1])):
        mat[index + 1][character[index + 1][i]] = {}
        for j in range(len(character[index + 1])):
            mat[index + 1][character[index + 1][i]][character[index + 1][j]] = row[i][j]

co_mat = pd.DataFrame.from_dict(mat[1])
co_mat.head()

,柯景騰,沈佳儀,阿和,許博淳,廖英宏
廖英宏,9.006612002849578857e-02,8.727215975522994995e-02,4.625630006194114685e-02,1.164279207587242126e-01,0.000000000000000000e+00
柯景騰,0.000000000000000000e+00,-1.266602575778961182e-01,-1.096525341272354126e-01,-6.924734264612197876e-02,9.006612002849578857e-02
沈佳儀,-1.266602575778961182e-01,0.000000000000000000e+00,-1.938269101083278656e-02,8.850599825382232666e-02,8.727215975522994995e-02
許博淳,-6.924734264612197876e-02,8.850599825382232666e-02,-1.539127379655838013e-01,0.000000000000000000e+00,1.164279207587242126e-01
阿和,-1.096525341272354126e-01,-1.938269101083278656e-02,0.000000000000000000e+00,-1.539127379655838013e-01,4.625630006194114685e-02


In [106]:
co_mat2 = pd.DataFrame.from_dict(mat[2])
co_mat2.head()

,阿拓,思螢,老板娘,阿不思
思螢,5.788053199648857117e-02,0.000000000000000000e+00,4.774375259876251221e-02,-4.589994624257087708e-02
老板娘,-4.693402349948883057e-02,4.774375259876251221e-02,0.000000000000000000e+00,2.362870723009109497e-01
阿不思,7.771591842174530029e-02,-4.589994624257087708e-02,2.362870723009109497e-01,0.000000000000000000e+00
阿拓,0.000000000000000000e+00,5.788053199648857117e-02,-4.693402349948883057e-02,7.771591842174530029e-02
